In [ ]:
import preprocessing

import numpy as np
from nilearn import surface, datasets
import networkx as nx
from fugw.scripts import lmds
import pickle

In [ ]:
generate_embedding = True
k = 30
n_landmarks = 2048

In [ ]:
if generate_embedding:  # this might take long
    fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage')
    coordinates, triangles = surface.load_surf_mesh(fsaverage.pial_left)
    
    G = nx.Graph()
    for (i, j, k) in triangles:
        G.add_edge(i, j, weight=np.linalg.norm(coordinates[i] - coordinates[j]))
        G.add_edge(j, k, weight=np.linalg.norm(coordinates[j] - coordinates[k]))
        G.add_edge(i, k, weight=np.linalg.norm(coordinates[i] - coordinates[k]))
    graph = nx.to_scipy_sparse_array(G, weight='weight', format='csr')
    
    geom_lmds = lmds.compute_lmds(
      coordinates,
      triangles,
      n_landmarks=n_landmarks,
      k=k,
      n_jobs=32,
      verbose=True,
    ).cpu().numpy()
    
    with open(f"lmds_{k}_{n_landmarks}.pkl", "wb") as fout:
        pickle.dump((geom_lmds, geom_lmds), fout)

In [ ]:
data_1, data_2, (coor, tri), _meta = \
    preprocessing.get_brain_data(None, embedding_path=f"lmds_{k}_{n_landmarks}.pkl")